In [381]:
import kan
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

directory_path="kaggle_dataset"

In [431]:
x_train = pd.read_parquet(f"{directory_path}/X_train_preprocessed.parquet")
##x_test = pd.read_parquet(f"{directory_path}/X_test_preprocessed.parquet")
y_train = pd.read_parquet(f"{directory_path}/y_train.parquet")


print(y_train.head())
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler([0,1])
x_train = scaler.fit_transform(x_train)
##x_test = scaler.transform(x_test)
## split the data into training and validation
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.01)


## convert everything to torch tensor
x_train = torch.tensor(x_train, dtype=torch.float32)
x_val = torch.tensor(x_val, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).squeeze()
y_train = torch.nn.functional.one_hot(y_train.long(), num_classes=2).float().view(-1,2)
y_val = torch.tensor(y_val.values, dtype=torch.float32).squeeze()
y_val = torch.nn.functional.one_hot(y_val.long(), num_classes=2).float().view(-1,2)

## create the dataset
dataset={}
dataset['train_input'] = x_train
dataset['train_label'] = y_train
dataset['test_input'] = x_val
dataset['test_label'] = y_val



   Depression
0           0
1           1
2           1
3           1
4           0


In [432]:
y_val[0:10]



tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]])

In [433]:
for k,v in dataset.items():
    print(type(v))


<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>


In [476]:
model=kan.KAN(width=[x_train.shape[1],10,2], grid=5, k=5,device="cpu")

checkpoint directory created: ./model
saving model version 0.0


In [477]:
for __ in range(4):
    _ = model.fit(dataset, opt="LBFGS", steps=250,loss_fn=torch.nn.BCEWithLogitsLoss(),batch=1024)
    y_pred = model(dataset['test_input'])
    y_pred=torch.sigmoid(y_pred)
    y_pred_binary=torch.argmax(y_pred,dim=1)
    y_val_binary=torch.argmax(y_val,dim=1)
    ## calcoliamo la precisione

    accuracy=(y_pred_binary==y_val_binary).float().mean()
    print(f"{accuracy=:.4f}")

| train_loss: 4.02e-01 | test_loss: 4.10e-01 | reg: 1.32e+02 | : 100%|█| 250/250 [01:33<00:00,  2.68


saving model version 0.1
accuracy=0.9296


| train_loss: 3.81e-01 | test_loss: 4.06e-01 | reg: 1.31e+02 | : 100%|█| 250/250 [01:32<00:00,  2.69


saving model version 0.2
accuracy=0.9397


| train_loss: 3.94e-01 | test_loss: 3.95e-01 | reg: 1.15e+02 | : 100%|█| 250/250 [01:30<00:00,  2.77


saving model version 0.3
accuracy=0.9404


| train_loss: 3.93e-01 | test_loss: 3.92e-01 | reg: 1.12e+02 | : 100%|█| 250/250 [01:31<00:00,  2.72

saving model version 0.4
accuracy=0.9390


In [464]:
y_pred = model(dataset['test_input'])
y_pred=torch.sigmoid(y_pred)

y_pred[0:10]


tensor([[9.9982e-01, 1.8790e-04],
        [9.9848e-01, 1.6223e-03],
        [9.1366e-01, 1.1097e-01],
        [9.9765e-01, 3.1983e-03],
        [9.9759e-01, 3.6842e-03],
        [2.1611e-02, 9.7681e-01],
        [9.8853e-01, 1.4761e-02],
        [9.9978e-01, 2.9554e-04],
        [9.9944e-01, 5.6188e-04],
        [9.7222e-01, 3.8844e-02]], grad_fn=<SliceBackward0>)

In [465]:
## calcoliamo quanti 0 e quanti 1 ci sono nel dataset di train
n_ones=y_train.sum()
n_zeros=y_train.shape[0]-n_ones

In [466]:
n_ones,n_zeros

(tensor(137900.), tensor(0.))

In [467]:
weight=torch.tensor([n_zeros/y_train.shape[0],n_ones/y_train.shape[0]])



In [468]:
threshold=1
y_pred_binary=torch.argmax(y_pred,dim=1)
y_val_binary=torch.argmax(y_val,dim=1)
## calcoliamo la precisione

accuracy=(y_pred_binary==y_val_binary).float().mean()
accuracy


tensor(0.9296)